<a href="https://colab.research.google.com/github/thepbordin/tflite-finetuning/blob/main/Fruit_Dataset_TensorFlow_Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libraries

In [ ]:
!pip install tflite-model-maker-nightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 643 kB 3.2 MB/s 
     |████████████████████████████████| 87 kB 6.6 MB/s 
     |████████████████████████████████| 596 kB 56.4 MB/s 
     |████████████████████████████████| 840 kB 57.0 MB/s 
     |████████████████████████████████| 128 kB 68.5 MB/s 
     |████████████████████████████████| 10.9 MB 9.3 MB/s 
     |████████████████████████████████| 1.1 MB 57.7 MB/s 
     |████████████████████████████████| 60.1 MB 110 kB/s 
     |████████████████████████████████| 1.3 MB 42.4 MB/s 
     |████████████████████████████████| 238 kB 10.0 MB/s 
     |████████████████████████████████| 3.4 MB 43.8 MB/s 
     |████████████████████████████████| 25.3 MB 2.1 MB/s 
     |████████████████████████████████| 99 kB 5.0 MB/s 
     |████████████████████████████████| 1.1 MB 15.9 MB/s 
     |████████████████████████████████| 352 kB 49.2 MB/s 
     |██████████████████████████████

In [ ]:
# !pip uninstall -y tensorflow && pip install -q tensorflow==2.7.0
# !sudo apt -y install libportaudio2
# !pip install scann
# !pip uninstall -y protobuf && pip install -q protobuf==3.19
# !pip uninstall -y folium && pip install -q folium==0.2.1
# !pip uninstall -y packaging && pip install -q packaging==20.9
# !pip uninstall -y tensorflow && pip install -q tensorflow==2.9.0
# !pip install -q --use-deprecated=legacy-resolver tflite-model-maker
# !pip install -q pycocotools
# !pip install -q opencv-python-headless==4.1.2.30
# !apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

In [ ]:
# Importing libraries
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install kaggle
! mkdir -p ~/.kaggle
! cp "/content/drive/MyDrive/API Keys/kaggle.json" ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!kaggle datasets download -d mbkinaci/fruit-images-for-object-detection

 60% 17.0M/28.4M [00:00<00:00, 90.1MB/s]
100% 28.4M/28.4M [00:00<00:00, 112MB/s] 


In [ ]:
# More libraries
import glob
import shutil
import xml.etree.cElementTree as ET
import pandas as pd
import random

In [ ]:
!unzip -q "/content/fruit-images-for-object-detection.zip" -d "/content/data"


In [ ]:
shutil.rmtree('data')

In [ ]:
for src in [{"type": "train",
             "path": "data/train_zip/train"},
            {"type": "test",
             "path": "data/test_zip/test"}]:
  for pth in glob.glob(f"{src['path']}/*.xml"):
    dest = f"/content/data/{src['type']}"
    os.makedirs(dest, exist_ok=True)
    shutil.move(pth, dest)
  for pth in glob.glob(f"{src['path']}/*.jpg"):
    dest = f"/content/data/{src['type']}"
    os.makedirs(dest, exist_ok=True)
    shutil.move(pth, dest)
shutil.rmtree("data/train_zip")
shutil.rmtree("data/test_zip")

In [ ]:
# Parse info from the files
valid_folders = glob.glob('/content/data/*')
valid_fnames = []
dest = "data/prp"
os.makedirs(dest, exist_ok=True)
for folder in valid_folders:
  for file in glob.glob(f'{folder}/*'):
      shutil.copyfile(
          file,
          f'{dest}/{file.split("/")[-1]}'
        )
      valid_fnames.append(file)
random.shuffle(valid_fnames)

df = pd.DataFrame(columns=["type", "filename", "name", "left", "top", "right", "top", "right", "bottom", "left", "bottom"])

# Looping through all of the files
for i, file in enumerate(valid_fnames):
  # Checks if it is an xml file or not
  try:
    # Parse into the lists
    tree = ET.parse(file)
    root = tree.getroot()
    filename = ''.join(list(file.split(".")[:-1]) + [".jpg"])
    name = root[6][0].text
    width = int(root[4][0].text)
    height = int(root[4][1].text)
    left = int(root[6][4][0].text) / width
    top = int(root[6][4][1].text) / height
    right = int(root[6][4][2].text) / width
    bottom = int(root[6][4][3].text) / height

    if i < 0.8 * len(valid_fnames):
      df.loc[i]=["TRAINING", filename, name, left, top, right, top, right, bottom, left, bottom]
    elif i < 0.9* len(valid_fnames):
      df.loc[i]=["VALIDATION", filename, name, left, top, right, top, right, bottom, left, bottom]
    else:
      df.loc[i]=["TEST",filename, name, left, top, right, top, right, bottom, left, bottom]
  except:
    continue

In [ ]:
df

,type,filename,name,left,top,right,top,right,bottom,left,bottom
0,TRAINING,/content/data/test/apple_82.jpg,apple,0.156250,0.259574,0.877404,0.259574,0.877404,0.859574,0.156250,0.859574
1,TRAINING,/content/data/train/apple_36.jpg,apple,0.048193,0.099415,0.933735,0.099415,0.933735,0.941520,0.048193,0.941520
3,TRAINING,/content/data/train/apple_66.jpg,apple,0.057500,0.098333,0.496250,0.098333,0.496250,0.720000,0.057500,0.720000
4,TRAINING,/content/data/train/banana_47.jpg,banana,0.008571,0.231939,0.977143,0.231939,0.977143,1.000000,0.008571,1.000000
8,TRAINING,/content/data/train/orange_54.jpg,orange,0.008000,0.074000,0.680000,0.074000,0.680000,0.800000,0.008000,0.800000
...,...,...,...,...,...,...,...,...,...,...,...
587,TEST,/content/data/train/banana_74.jpg,banana,0.090278,0.037500,0.962500,0.037500,0.962500,0.400000,0.090278,0.400000
588,TEST,/content/data/test/apple_90.jpg,apple,0.031974,0.062000,0.979052,0.062000,0.979052,0.955000,0.031974,0.955000
593,TEST,/content/data/train/orange_36.jpg,orange,0.441250,0.214167,0.900000,0.214167,0.900000,0.815000,0.441250,0.815000
596,TEST,/content/data/train/orange_32.jpg,orange,0.125000,0.334471,0.620000,0.334471,0.620000,0.986348,0.125000,0.986348


In [ ]:
pd.unique(df["name"])

array(['apple', 'banana', 'orange'], dtype=object)

In [ ]:
# Turns it into the csv
df.to_csv('data.csv', index=False)

In [ ]:
# Deletes the first line
with open("test.csv",'r') as f, open("trainingdata.csv",'w') as f1:
    next(f) # skip header line
    for line in f:
        f1.write(line)

In [ ]:
# Loads training data
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('trainingdata.csv')

In [ ]:
# Initialize model
spec = model_spec.get('efficientdet_lite0')

In [ ]:
# Starts training
model = object_detector.create(train_data, model_spec=spec, batch_size=32, train_whole_model=True, validation_data=validation_data)

Epoch 1/50
6/6 [==============================] - 62s 3s/step - det_loss: 1.8155 - cls_loss: 1.1522 - box_loss: 0.0133 - reg_l2_loss: 0.0632 - loss: 1.8787 - learning_rate: 0.0232 - gradient_norm: 0.9373 - val_det_loss: 1.7173 - val_cls_loss: 1.1292 - val_box_loss: 0.0118 - val_reg_l2_loss: 0.0632 - val_loss: 1.7805
Epoch 2/50
6/6 [==============================] - 9s 2s/step - det_loss: 1.6603 - cls_loss: 1.1160 - box_loss: 0.0109 - reg_l2_loss: 0.0632 - loss: 1.7235 - learning_rate: 0.0399 - gradient_norm: 0.8079 - val_det_loss: 1.5082 - val_cls_loss: 1.0366 - val_box_loss: 0.0094 - val_reg_l2_loss: 0.0632 - val_loss: 1.5714
Epoch 3/50
6/6 [==============================] - 10s 2s/step - det_loss: 1.4822 - cls_loss: 1.0131 - box_loss: 0.0094 - reg_l2_loss: 0.0632 - loss: 1.5454 - learning_rate: 0.0397 - gradient_norm: 1.0139 - val_det_loss: 1.3073 - val_cls_loss: 0.9013 - val_box_loss: 0.0081 - val_reg_l2_loss: 0.0632 - val_loss: 1.3705
Epoch 4/50
6/6 [==============================]

In [ ]:
# Testing the model
model.evaluate(test_data)

1/1 [==============================] - 6s 6s/step



{'AP': 0.6739385,
 'AP50': 0.91131747,
 'AP75': 0.7388519,
 'AP_/apple': 0.9020745,
 'AP_/banana': 0.40847507,
 'AP_/orange': 0.7112661,
 'APl': 0.6739385,
 'APm': -1.0,
 'APs': -1.0,
 'ARl': 0.79136366,
 'ARm': -1.0,
 'ARmax1': 0.6780303,
 'ARmax10': 0.79136366,
 'ARmax100': 0.79136366,
 'ARs': -1.0}

Load a model in and predict

In [ ]:
model.export(export_dir='.')

In [ ]:
import cv2

from PIL import Image

model_path = 'model.tflite'

# Load the labels into a list
classes = {
    0: "apple",
    1: "banana",
    2: "orange",
}

# Code from tensorflow's website
# Define a list of colors for visualization
COLORS = np.random.randint(0, 255, size=(5, 3), dtype=np.uint8)

def preprocess_image(image, input_size):
  """Preprocess the input image to feed to the TFLite model"""
  img = tf.convert_to_tensor(image, dtype=tf.float32)
  original_image = img
  resized_img = tf.image.resize(img, input_size)
  resized_img = resized_img[tf.newaxis, :]
  resized_img = tf.cast(resized_img, dtype=tf.uint8)
  return resized_img, original_image


def detect_objects(interpreter, image, threshold):
  """Returns a list of detection results, each a dictionary of object info."""

  signature_fn = interpreter.get_signature_runner()

  # Feed the input image to the model
  output = signature_fn(images=image)

  # Get all outputs from the model
  count = int(np.squeeze(output['output_0']))
  scores = np.squeeze(output['output_1'])
  classes = np.squeeze(output['output_2'])
  boxes = np.squeeze(output['output_3'])

  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
        'bounding_box': boxes[i],
        'class_id': classes[i],
        'score': scores[i]
      }
      results.append(result)
  return results


def run_odt_and_draw_results(image, interpreter, threshold=0.5):
  """Run object detection on the input image and draw the detection results"""
  # Load the input shape required by the model
  _, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

  # Load the input image and preprocess it
  preprocessed_image, original_image = preprocess_image(
      image,
      (input_height, input_width)
    )

  # Run object detection on the input image
  results = detect_objects(interpreter, preprocessed_image, threshold=threshold)

  # Plot the detection results on the input image
  original_image_np = original_image.numpy().astype(np.uint8)
  for obj in results:
    # Convert the object bounding box from relative coordinates to absolute
    # coordinates based on the original image resolution
    ymin, xmin, ymax, xmax = obj['bounding_box']
    xmin = int(xmin * original_image_np.shape[1])
    xmax = int(xmax * original_image_np.shape[1])
    ymin = int(ymin * original_image_np.shape[0])
    ymax = int(ymax * original_image_np.shape[0])

    # Find the class index of the current object
    class_id = int(obj['class_id'])

    # Draw the bounding box and label on the image
    color = [int(c) for c in COLORS[class_id]]
    cv2.rectangle(original_image_np, (xmin, ymin), (xmax, ymax), color, 2)
    # Make adjustments to make the label visible for all objects
    y = ymin - 15 if ymin - 15 > 15 else ymin + 15
    label = "{}: {:.0f}%".format(classes[class_id], obj['score'] * 100)
    cv2.putText(original_image_np, label, (xmin, y),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

  # Return the final image
  original_uint8 = original_image_np.astype(np.uint8)
  return original_uint8

In [ ]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=86fd6f3550a5256346ce529195497664bc501931473a068ba2bf55e7e70277d3
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [ ]:
#@title Default title text
# INPUT_IMAGE_URL = "https://www.istockphoto.com/photos/fruit-isolated" #@param {type:"string"}
DETECTION_THRESHOLD = 0.1 #@param {type:"slider", min:0.1, max:0.9, step:0.1}
import PIL

arr = np.array(PIL.Image.open('/content/img.jpg'))

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Run inference and draw detection result on the local copy of the original file
detection_result_image = run_odt_and_draw_results(
    arr,
    interpreter,
    threshold=DETECTION_THRESHOLD
)

# Show the detection result
(detection_result_image)

NameError: ignored